
<center>
    <h1> INF285/ILI285 Computación Científica </h1>
    <h1> Tarea N°1, v1.00 </h1>
    
</center>

<p>

</p>

Gabriel Carmona
201773509-0

## Instrucciones

* La tarea es individual.
* Las consultas sobre las tareas se deben realizar por medio de la plataforma Aula.
* La tarea debe ser realizada en `Jupyter Notebook` (`Python3`).
* Se evaluará la correcta utilización de librerias `NumPy`, `SciPy`, entre otras, así como la correcta implementación de algoritmos de forma vectorizada.
*  **El archivo de entrega debe denominarse ROL-tarea-numero.ipynb**. _De no respetarse este formato existirá un descuento de **50 puntos**_
* La fecha de entrega es el jueves 30 de Abril a las **18:00 hrs**.  Se aceptarán entregas hasta las 19:00 hrs sin descuento en caso de existir algun problema, posteriormente existirá un descuento lineal hasta las 20:00 hrs del mismo día.
* Las tareas que sean entregadas antes del jueves a mediodía recibirán una bonificación de 10 puntos
* Debe citar cualquier código ajeno utilizado (incluso si proviene de los Jupyter Notebooks del curso).




In [2]:
from math import sqrt
import numpy as np
from scipy.special import legendre
import scipy.special as sp

## Introducción 

En esta primera tarea de INF/ILI-285, versión 2020-1, estudiaremos la importancia de los primeros temas estudiados en el curso, los cuales son: Representación de Punto Flotante, Pérdida de Significancia, Errores de Cancelación y Búsqueda de Ceros. El desarrollo de cada uno de esos temas se presenta en una serie de preguntas, donde ustedes deben ir decidiendo, pregunta a pregunta, como cada uno de los temas se aplica. En general, los temas no se analizan por separado, sino de manera acoplada. Es muy importante que cada uno de los problemas sea analizado teóricamente primero, para luego poner su conocimiento en acción. Cada problema puede ser desarrollado de diversas formas, sin embargo, es muy importante determinar al final si el camino elegido resuelve la pregunta presentada.

## Problemas

### 1. Simulador (50 ptos) 

Dada la familia de polinomios de grado 3 con la forma:


\begin{equation}
f(x)=x^3 - A\,x^2 + A\,x - 1
\end{equation}


Se pide implementar un algoritmo que reciba como parámetros el valor de $A$ (con $|A|>>1$) y la cantidad de bits que tiene en la mantisa un computador ficticio, el manejo del exponente debe considerarse de las misma forma que lo maneja _double precision_.
Esta implementación debe calcular las raíces de $f$ sin perdida de significancia con la cantidad de bits disponibles para la mantisa.
Para obtener las raíces de $f$ usted debe encontrar de forma algebraica sus raíces y luego proponer un algoritmo basado en las _fórmulas_ obtenidas.

Considere que en ese computador ficticio cuenta con las operaciones matemáticas necesarias para obtener las raíces.  Considere el siguiente ejemplo:

```python
# Alg. Base
a = 9819824,624837
b = 148736.523476
c = a+b

# Alg. con Representación de Punto Flotante de 'bits_mant' bits en la mantisa.
am = f_new_rep(9819824.624837,bits_mant) # Aproximar el input en la nueva representación.
bm = f_new_rep(148736.523476,bits_mant) # Aproximar el input en la nueva representación.
cm = f_new_rep(m,exp,am+bm) # Aproximar el output de la suma y cada operación en la nueva representación.
```


En una primera instancia se define el código para poder pasar cualquier número a la nueva representación.

Primero, se utiliza la función partir_en_dos para obtener la parte entera y la parte decimal del número que me dieron. Lo que se hace es partir básicamente el número en 2 y devolver una lista donde la primera posición será la parte entera y la segunda posición será la parte decimal del númer. Esto se hace puesto que para pasar el a binario se utilizan métodos distintos para la parte entera y decimal.

In [3]:
"""
input
x      : (double) número a partir en dos
output
lista  : (lista) lista de dos elementos, la parte entera 
y parte decimal del número x
"""
def partir_en_dos(x):
    if("e" not in str(x)): #para verificar que no sea un número muy chico o muy grande
        x = float(x)
        return str(x).split(".")
    else:
        num, potencia = str(x).split("e")
        if("." in num):
            numEntero, numDecimal = num.split(".")
        else:
            numEntero = num
            numDecimal = ""
        if("-" == potencia[0]):
            i = 0
            numToReturn = ""
            while i < int(potencia[1:]):
                if i < int(potencia[1:]) - 1:
                    numToReturn += "0"
                else:
                    numToReturn = numToReturn + numEntero + numDecimal
                i += 1
            return ["0",numToReturn]
        else:
            i = 0
            numToReturn = numEntero + numDecimal
            
            while i < int(potencia[1:]) - len(numDecimal):
                numToReturn += "0"
                i += 1 
            return [numToReturn,"0"]

Luego de recibir la parte entera y decimal utilizamos las siguientes dos funciones. Primero para pasar un entero a binario, el cual utiliza el método de preguntar si es el número es divisible por 2 o no [1]:

1. Si es divisible agrego un 0
2. Si no es divisible agrego un 1 


Luego, se divide el número por 2 y se aplica el mismo proceso hasta que el número sea menor a 0.

Finalmente se invierte la lista resultante y se une la lista para retorna el string del entero en binario. 

In [4]:
"""
input
x         : (int) número a pasar a binario
output
xBin      : (string) binario del numero dado
"""
def bin_int(x):
    xBin = []
    while x >= 1:
        xBin.append("1" if x % 2 else "0")
        x = int(x / 2)
    xBin.reverse()
    xBin = "".join(xBin)
    return xBin

Para la parte decimal, lo que se realiza es multiplicar por 2 el número recibido, luego se pregunta si el número es mayor o menor 1. Si es mayor a 1, entonces se agrega un 1 al string resultante y solo nos quedamos con la parte decimal. Si es menor a 1, entonces se agrega un 0 al string resultado [1]. Este proceso se repite hasta que 2 situaciones pasen:


1. Si el número da exactamente 1, entonces se agrega el bit 1 y se hace break.

2. Si ya ha agregrado la cantidad de bits de mantisa que me dan, sigo iterando hasta que el número se vuelva 1 eventualmente o haya encontrado dos 1. Esto me va a servir para el redondeo sin tener que preocuparme por numeros infinitos.

Luego de terminar este loop, retornamos el string resultante.

In [5]:
"""
input
x         : (double) número a pasar a binario
bits_mant : (int) cantidad de bits de mantisa
output
xBin      : (string) binario del numero dado
"""
def bin_float(x, bits_mant):
    xBin = ""
    patronsito = []
    iterador = 0
    cantUnoPostMantisa = 0
    encontreUno = 0
    while x <= 1:
        xBin += "1" if int(x * 2.0) else "0"
        if(xBin[-1] == "1" and encontreUno == 0):
            encontreUno = 1
        elif(encontreUno):
            iterador += 1
        if(x * 2.0 == 1.0):
            break
        if(iterador > bits_mant):
                if(xBin[-1] == "1"):
                    cantUnoPostMantisa += 1
                if(cantUnoPostMantisa > 1):
                    break
        x = x * 2.0
        
        if("e" in str(x)):
            x = float("0." + partir_en_dos(x)[1])
        else:
            x = float("0."+ str(x).split(".")[1])
        
    return xBin

Luego de tener ambos binarios, los juntamos realizamos el shif correspondiente para dejarlo en notación cientifica y se le pasa a la función aproximar. Esta función básicamente sigue los estandares de la IEEE vistos en clases y mostrados por el libro [1] utilizando claramente la mantisa dada.

In [6]:
"""
input
xBin      : (string) binario a aproximar
bits_mant : (int) cantidad de bits de mantisa
output
retorna el binario aproximado o n, en el caso 
de que el binario dado sea puros 1, para luego sumarle uno al exopnente
"""
def aproximar(xBin, bits_mant):
    if(xBin[bits_mant] == "0"):
        return "".join(xBin[0:bits_mant])
    else:
        if(len(xBin) == bits_mant + 1):
            if xBin[len(xBin) - 1] == "1":
                if "0" in xBin[0:bits_mant]:
                    i = bits_mant
                    xBinTemp = []
                    for x in xBin:
                        xBinTemp.append(x)
                    xBin = xBinTemp
                    while(i >= 0):
                        if(xBin[i] == "1"):
                            xBin[i] = "0"
                        else:
                            xBin[i] = "1"
                            break
                        i -= 1
                    return "".join(xBin[0:bits_mant])
                else:
                    return "n"
            return "".join(xBin[0:bits_mant])
        else:
            if "0" in xBin[0:bits_mant]:
                i = bits_mant
                xBinTemp = []
                for x in xBin:
                    xBinTemp.append(x)
                xBin = xBinTemp
                while i >= 0:
                    if(xBin[i] == "1"):
                        xBin[i] = "0"
                    else:
                        xBin[i] = "1"
                        break
                    i -= 1
                return "".join(xBin[0:bits_mant])
            else:
                return "n"

Finalmente, se pasa el número de binario a número se aplica lo de $(2^0+2^{-1}+...+2^{-(bits\_mant - 1)}+2^{-bits\_mant})\cdot 2^{exp}$ [1]. Y se retorna la suma.

In [7]:
"""
input
x      : (string) binario a pasar a número
exp    : (int) exponente a multiplicar el binario
output
suma   : (double) suma resultante de pasar de binario a número
"""
def binario_a_numero(x, exp):
    i = -1
    suma = 1.0
    for b in x:
        suma += int(b) * 2 ** i
        i -= 1
    suma = suma * 2** exp
    return suma

Esta es la gran función que utiliza todoas las funciones anteriores y aplica el shift mencionado, dependiendo si existe parte entera o no.

In [8]:
"""
input
x         : (double) valor a evaluar
bits_mant : (int) cantidad de bits de la mantisa
output
eval      : (double) resultado obtenido
"""

def f_new_rep(x, bits_mant):
    # Algoritmo de representación de punto flotante modificada.
    if(x == 0.0):
        return 0
    # Obtener signo
    xSigno = -1 if x < 0 else 1
    x = abs(x)
    # Obtener parte entera y parte decimal
    xEntero, xDecimal = partir_en_dos(x)
        
    xEntero = int(xEntero)
    xDecimal = float("0." + xDecimal)
    #print("----Parte Entera:", xEntero, "ParteDecimal", xDecimal)
    
    # Pasar a binarios ambos valores
    xEntero = bin_int(xEntero)
    #print("----Parte Entera Bin:", xEntero)
    xDecimal = bin_float(xDecimal, bits_mant) if xDecimal != 0.0 else "0"
    #print("----Parte Decimal Bin:", xDecimal)
    print(xEntero +"."+xDecimal)
    # Shiftear a notacion cientifica y obtener exponente
    if len(xEntero) == 0:
        i = 0
        for x in xDecimal:
            if(x == "1"):
                break
            i+=1
        expo = 0 - (i + 1)
        finalBin = xDecimal[i + 1:len(xDecimal)]
    elif len(xEntero) == 1:
        finalBin = xDecimal
        expo = 0
    else:
        finalBin = xEntero[1:len(xEntero)] + xDecimal
        expo = len(xEntero) - 1
        
    #print("----Shifteado",finalBin)
    
    # Aproximar si la cantidad de bits es mayor a la de la mantisa
    if(len(finalBin) > bits_mant):
        finalBin = aproximar(finalBin, bits_mant)
    if(finalBin == "n"):
        expo += 1
        finalBin = ""
    #print("----Signo", xSigno)
    #print("----Exponente", expo)
    #print("----Final ahora si que si", finalBin)
    return xSigno * binario_a_numero(finalBin, expo)

In [13]:
f_new_rep(23.375,64)

10111.011


23.375

La función del principio podemos ver que x = 1, entonces sabemos que podemos dividir (x-1) con la función f para poder terminar las otras raices, el cual nos entrega la siguiente cuadrática.

\begin{equation}
f(x)=x^3 - A\,x^2 + A\,x - 1 = (x - 1)(x^2 + (1 - A)x + 1)
\end{equation}

\begin{equation}
(x - 1)(x^2 + (1 - A)x + 1) = 0
\begin{cases}
x_1 = 1\\
x_2 = \frac{A - 1 - \sqrt{(1 - A)^2 - 4}}{2}\\
x_3 = \frac{A - 1 + \sqrt{(1 - A)^2 - 4}}{2}
\end{cases}
\end{equation}

Ahora para esta ecuación, hay que transformarla para poder evitar la perdida de significancia, esta transformación dependerá de si es que el A es negativo o es positivo. 

Para el caso donde el A es positivo, realizamos la transformación solo en $x_{2}$ puesto que en este valor sucede la resta, quedando lo siguiente:

\begin{equation}
x_2 = \frac{A - 1 - \sqrt{(1 - A)^2 - 4}}{2}\cdot \frac{A - 1 + \sqrt{(1 - A)^2 - 4}}{A - 1 + \sqrt{(1 - A)^2 - 4}}
\end{equation}
\begin{equation}
x_2 = \frac{2}{A - 1 + \sqrt{(A - 1)^2 - 4}}
\end{equation}

\begin{equation}
x_3 = \frac{A - 1 + \sqrt{(A - 1)^2 - 4}}{2}
\end{equation}

Ahora si es que el A es negativo, realizamos la transformación solo en $x_{3}$ puesto que en este valor sucederá la resta, quedando lo siguiente:

\begin{equation}
x_2 = \frac{A - 1 - \sqrt{(1 - A)^2 - 4}}{2}
\end{equation}

\begin{equation}
x_3 = \frac{A - 1 + \sqrt{(A - 1)^2 - 4}}{2}\cdot \frac{A - 1 - \sqrt{(1 - A)^2 - 4}}{A - 1 - \sqrt{(1 - A)^2 - 4}}
\end{equation}
\begin{equation}
x_2 = \frac{2}{A - 1 - \sqrt{(A - 1)^2 - 4}}
\end{equation}

Por cada operación realizada a almacenar, se pasa a la nueva representación.

In [26]:
def f_find_roots(A,bits_mant):
    A = float(A)
    # Algoritmo para encontrar raíces de f(x).
    #print("Raiz 1:",1)
    #print()
    x_roots = [1]
    Aminus1 = f_new_rep(A - 1, bits_mant)
    #print("A - 1", Aminus1)
    Aminus12 = f_new_rep(Aminus1 * Aminus1, bits_mant)
    #print("(A - 1)^2",Aminus12)
    Aminus12minus4 = f_new_rep(Aminus12 - 4, bits_mant)
    #print("(A - 1)^2 - 4",Aminus12minus4)
    raizAminus12minus4 = f_new_rep(sqrt(Aminus12minus4), bits_mant)
    #print("raiz((A - 1)^2 - 4)",raizAminus12minus4)
    if(A > 0):
        Aminus1plusRaiz = f_new_rep(Aminus1 + raizAminus12minus4, bits_mant)
    #    print("A - 1 + raiz((A - 1)^2 - 4)", Aminus1plusRaiz)
        x_roots.append(f_new_rep(2.0 / Aminus1plusRaiz, bits_mant))
    #    print("Raiz 2:", x_roots[1])
        x_roots.append(f_new_rep(Aminus1plusRaiz / 2.0, bits_mant))
    #   print("Raiz 3:", x_roots[2])
    else:
        Aminus1minusRaiz = f_new_rep(Aminus1 - raizAminus12minus4, bits_mant)
    #    print("A - 1 - raiz((A - 1)^2 - 4)", Aminus1minusRaiz)
        x_roots.append(f_new_rep(Aminus1minusRaiz / 2.0, bits_mant))
    #    print("Raiz 2:", x_roots[1])
        x_roots.append(f_new_rep(2.0 / Aminus1minusRaiz, bits_mant))
    #    print("Raiz 3:", x_roots[2])
    return x_roots

### Polinomios de Legendre (50 puntos)
Dada la función compuesta $f$
\begin{equation}
f_{n,m}(x) = L_n(C_m(x)),
\end{equation}
donde $L_n$ es conocido como el polinomio de Legendre de grado $n$ definido de la siguiente forma:
\begin{equation}
L_{n}(x)=\frac{1}{2^{n}} \sum_{k=0}^{n}\left(\begin{array}{l}
n \\
k
\end{array}\right)^{2}(x-1)^{n-k}(x+1)^{k},
\end{equation}
y $C_m$ es el polinomio de Chebyshev
\begin{equation}
C_m(x) = \cos(m \cdot \arccos(x)).
\end{equation}

Utilizando el método de Bisección y Punto fijo se pide obtener la raíz de $f$ más cercana a $0.5$ dado un valor de $m$ y $n$



*Hint: Las raíces de Legendre son conocidas*


Definimos una función para determinar la raiz más cercana a 0.5, de un polinomio de legendre de grado n, esto nos servirá en el futuro para el método de iteración de punto fijo.

In [27]:
"""
input
n         : (int) el grado del polinomio al cual le voy a sacar las roots
output
masCercano: (double) retorna la raiz más cerca a 0.5 que tenga el polinomio 
de grado n
"""
def raiz_mas_cercana(n):
    raices = sp.roots_legendre(n)[0]
    masCercano = 1000
    for x in raices:
        if abs(x - 0.5) < abs(masCercano - 0.5):
            masCercano = x
    return masCercano

En el método de bisección lo que hacemos en tomar r = 0.5, luego lo que hacemos es definir dos intervalos $[a_1,b_1]$ y $[a_2,b_2]$, siendo $b_1 = 0.5$ y $a_2 = 0.5$. Luego empezamos a sumar un intervalo pequeñito hasta que cambie de signo claramente saltandome las respectivas raices. Y luego aplicamos bisección así tal cual para ambos intervalos siguiendo los pasos del libro [1].

Luego de obtener las dos posibles raices, vemos cual es la raiz más cercana a 0.5 y retornamos.

In [28]:
"""
input
n: (int) grado del polinomio de Legendre
m: (int) grado del polinomio de Chebyshev
tol: (double) tolerancia para la detención del algoritmo
output
root: (double) raiz obtenida
"""
def f_Biseccion(n, m, tol):
    r = 0.5
    
    aSumar = 1 / (m * 1e3)
    
    if sp.legendre(n)(sp.chebyt(m)(r)) == 0.0:
        return r
    
    aDer = r
    bIzq = r
    
    while sp.legendre(n)(sp.chebyt(m)(bIzq)) * sp.legendre(n)(sp.chebyt(m)(r)) > 0:
        r -= aSumar
    aIzq = r
    r = 0.5
    while sp.legendre(n)(sp.chebyt(m)(aDer)) * sp.legendre(n)(sp.chebyt(m)(r)) > 0:
        r += aSumar
    bDer = r
    
    cDer = (aDer + bDer) / 2
    while abs(sp.legendre(n)(sp.chebyt(m)(cDer))) >= tol:
        if sp.legendre(n)(sp.chebyt(m)(cDer)) * sp.legendre(n)(sp.chebyt(m)(aDer)) < 0:
            bDer = cDer
        else:
            aDer = cDer
        cDer = (aDer + bDer) / 2
    cIzq = (aIzq + bIzq) / 2
    while abs(sp.legendre(n)(sp.chebyt(m)(cIzq))) >= tol:
        if sp.legendre(n)(sp.chebyt(m)(cIzq)) * sp.legendre(n)(sp.chebyt(m)(aIzq)) < 0:
            bIzq = cIzq
        else:
            aIzq = cIzq
        cIzq = (aIzq + bIzq) / 2   
    if abs(cIzq - 0.5) <  abs(cDer - 0.5):
        return cIzq
    else:
        return cDer

Como sabemos que las raices del polinomio de Legendre, sabemos que 

\begin{equation}
C_m(x) = r
\end{equation}

Ahora bien, como siempre hay dos posibles raices cercanas a 0.5, a la izquierda y derecha de este definimos estas dos puntos de iteración:

La primera para encontrar la raiz a la izquierda

\begin{equation}
x = \frac{\log_{10}{(\cos(m \cdot \arccos(x)) - r + 10^{m\cdot x})}}{m}
\end{equation}

La segunda para encontrar la raiz a la derecha
\begin{equation}
x = \frac{\log_{10}{(r - \cos(m \cdot \arccos(x)) + 10^{m\cdot x})}}{m}
\end{equation}

Como initial guess utlizaremos $x_0 = 0.5$

In [46]:
def f_FPI(n, m, tol):
    r = raiz_mas_cercana(n)
    f = lambda x: np.log10(r-np.cos(m * np.arccos(x)) + 10**(m*x))/(m)
    f2 = lambda x: np.log10(np.cos(m * np.arccos(x)) - r + 10**(m*x))/(m)
    
    ig = 0.5
    root1 = ig
    print("ya")
    while 1:
        newRoot1 = f(root1)
        if abs(newRoot1 - root1) < tol:
            break
        root1 = newRoot1
    root2 = ig
    print("ya2")
    while 1:
        newRoot2 = f2(root2)
        if abs(newRoot2 - root2) < tol:
            break
        root2 = newRoot2
    if abs(root1 - 0.5) <  abs(root2 - 0.5):
        return root1
    else:
        return root2

In [47]:
f_Biseccion(7,7,0.0000001)

0.48686708232334464

In [50]:
n_values=[8,10]
m_values=[5,10]

for n in n_values:
    for m in m_values:
        print(n,m)
        print("bis")
        f_Biseccion(n,m,1e-10)
        print("FPI")
        f_FPI(n,m,1e-10)

8 5
bis
FPI
ya
ya2
8 10
bis
FPI
ya


KeyboardInterrupt: 

Despues de varias pruebas, pude ver que la bisección me funciono bastante bien, aunque el punto fijo siempre estuvo cerca, pero jamás tan cerca a valor igual, pero igual encontraba raices cerca de 0.5, pero no las más cerca.

###  Determinantes (20 puntos)

Dada una matriz de dimensiones $ n \times n$ de la forma:
\begin{equation}
A
=
\begin{pmatrix}
a_{1,1} & a_{1,2}  & \dots & a_{1,n} \\
a_{2,1} & a_{2,2}  & \dots & a_{2,n} \\
\vdots &  \vdots   & \ddots & \vdots \\
a_{n,1} & a_{n,2}  & \dots & a_{n,n}
\end{pmatrix}
=
\begin{pmatrix}
\mathbf{r}_1 \\
\mathbf{r}_2 \\
\vdots \\
\mathbf{r}_n \\
\end{pmatrix},
\end{equation}
donde $\mathbf{r}_k$ representa la $k$-ésima fila de la matriz $A$.
Considere la siguiente matriz $C_{i,j}(\alpha)$,
\begin{equation}
C_{i,j}(\alpha)
=
\begin{pmatrix}
\mathbf{r}_1 \\
\vdots \\
\mathbf{r}_i\,(1-\alpha)+\mathbf{r}_j\,\alpha \\
\vdots \\
\mathbf{r}_j\,(1-\alpha)+\mathbf{r}_i\,\alpha \\
\vdots \\
\mathbf{r}_n \\
\end{pmatrix},
\end{equation}
de lo cual sabemos que $C_{i,j}(0)=A$ y que $C_{i,j}(1)$ es la matriz $A$ donde se intercambiaron las filas $i$ y $j$, es decir:
\begin{equation}
C_{i,j}(1)
=
\begin{pmatrix}
\mathbf{r}_1 \\
\vdots \\
\mathbf{r}_j \\
\vdots \\
\mathbf{r}_i \\
\vdots \\
\mathbf{r}_n \\
\end{pmatrix}.
\end{equation}
De las relaciones anteriores podemos concluir que el determinante de la matriz $A$, denominado $D=\det(A)$, es igual al determinante de $C_{i,j}(0)$, es decir $\det(C_{i,j}(0))=\det(A)=D$.
Por el otro lado, el determinante de $C_{i,j}(1)$ es $-D$, dado que es el intercambio de las filas $i$ y $j$ de la matriz $A$.
Por lo cual podemos concluir que $-D\leq \det(C_{i,j}(\alpha))\leq D$.

Utilizando el método de Bisección debe encontrar el valor de $\alpha$ con $p$ decimales de precisión que permitan que, dado los índices de las filas $i$, $j$, y $i\neq j$, el determinante de la matriz sea igual a $d$, donde $d\in[-D,D]$. 

Para esto se debe implementar una función que reciba la matriz $A$, las filas $i$ y $j$, y $p$; y retorne $\widehat{\alpha}$ tal que $\det(C_{i,j}(\widehat{\alpha}))=d$.


Lo que se realiza en este problema, es el método de bisección clásico para buscar puntos fijos, donde nuestra función a evaluar sería $det(A(\alpha))$ dado un alfa. Y luego se trata de la misma forma la busqueda de bisección, pero para el valor de $d-det(A(\alpha))$. Esto permite responder al problema correctamente, con la precisión p que te dan (utilizandola como tolerancia)

In [31]:
"""
intput
A               : (array n x n) matriz a aplicar alpha
d               : (double) d buscado
alpha           : (double) alpha a evaluar la matriz 
i               : (int) indice de la fila i
j               : (int) indice de la fila j
output
d - det(A_copia): (double) valor de la diferencia entre d y el determinar de la 
matriz modificada
"""
def calcular_d_matriz(A, d, alpha, i, j):
    A_copia = np.copy(A)
    new = np.add(np.multiply(A_copia[i],(1 - alpha)), np.multiply(A_copia[j],alpha))
    new2 = np.add(np.multiply(A_copia[j],(1 - alpha)), np.multiply(A_copia[i],alpha))
    A_copia[i] = new
    A_copia[j] = new2
    return d - np.linalg.det(A_copia)

In [32]:
"""
input
A: (array n x n) matriz
i: (int) índice de la fila "i".
j: (int) índice de la fila "j".
p: (int) cantidad de decimales de precision 
d: (double) valor requerido del determinante de $C_{i,j}(\alpha)$, i.e. $\det(C_{i,j}(\widehat{\alpha}))=d$.
output
alpha_hat: (double) alpha_hat tal que det(C_{i,j}(alpha_hat))=d.
"""


def find_alpha_hat(A,i,j,p,d):
    # Algoritmo a desarrollar, note que usted debe definir el intervalo a utilizar en el método de la bisección.

    if abs(np.linalg.det(A)) <= abs(d):    
        return 1 if np.linalg.det(A) * d < 0 else 0
    a = 0
    b = 1
    alpha_hat = (a + b) / 2
    Dalpha = calcular_d_matriz(A, d, alpha_hat, i, j)
    n_iter = (np.log2((1 / (0.5 * 10**-p))) - 1) + 1
    h = 0
    while abs(Dalpha) >= 10**(-p) and n_iter > h:
        if(Dalpha * calcular_d_matriz(A, d, a, i, j) < 0):
            b = alpha_hat
        else:
            a = alpha_hat
        alpha_hat = (a + b) / 2.0
        
        Dalpha = calcular_d_matriz(A, d, alpha_hat, i, j)
        h += 1
    return round(alpha_hat, p)

# Referencias

1. Numerical Analysis Second Edition, Timothy Sauer, 2006 Pearson education.